In [1]:
# Load necessary libraries
library(keras)
library(tensorflow)
library(dplyr) # For data manipulation
library(imager)
library(caret)
library(e1071)
library(ggplot2)
library(EBImage)  # For bwlabel and shape feature extraction

# Set the path to your dataset
data_dir <- "/kaggle/input/leukemia-images/Original"

# Define image size and parameters
img_height <- 150
img_width <- 150
batch_size <- 32

# Create a list to hold all file paths and corresponding labels
file_paths <- c()
class_labels <- c()

# Loop through each class to gather file paths and labels
for (class in c("Benign", "Early", "Pre", "Pro")) {
  class_path <- file.path(data_dir, class)
  files <- list.files(class_path, full.names = TRUE)
  
  # Ensure files are unique
  unique_files <- unique(files)

  file_paths <- c(file_paths, unique_files)
  class_labels <- c(class_labels, rep(class, length(unique_files)))  # Store the class name
}

# Create a DataFrame with only the required structure
full_df <- data.frame(
  image_filename = file_paths,
  class = class_labels,
  stringsAsFactors = FALSE
)

head(full_df)


Attaching package: ‘dplyr’




The following objects are masked from ‘package:stats’:

    filter, lag




The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




Loading required package: magrittr




Attaching package: ‘imager’




...

The following object is masked from ‘package:tensorflow’:

    train




The following object is masked from ‘package:httr’:

    progress





Attaching package: ‘EBImage’




The following objects are masked from ‘package:imager’:

    channel, dilate, display, erode, resize, watershed




,image_filename,class
,<chr>,<chr>
1,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-001.jpg,Benign
2,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-002.jpg,Benign
3,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-003.jpg,Benign
4,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-004.jpg,Benign
5,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-005.jpg,Benign
6,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-006.jpg,Benign


In [2]:
# Define class-to-index mapping
classes <- c("Benign", "Early", "Pre", "Pro")
class_to_index <- setNames(0:(length(classes) - 1), classes)

# Add numeric columns to the full and training DataFrames
full_df$class_numeric <- as.numeric(factor(full_df$class, levels = classes)) - 1

# Stratified split into training (64%), validation (20%), and test (16%) sets
set.seed(123)
trainIndex <- createDataPartition(full_df$class, p = 0.64, list = FALSE)
train_df <- full_df[trainIndex, ]
remaining_df <- full_df[-trainIndex, ]

# Shuffle only train_df
train_df <- train_df[sample(nrow(train_df)), ]

# Split the remaining 36% into validation (20%) and test (16%)
validationIndex <- createDataPartition(remaining_df$class, p = 20 / (20 + 16), list = FALSE)
val_df <- remaining_df[validationIndex, ]
test_df <- remaining_df[-validationIndex, ]

head(train_df)
head(val_df)
head(test_df)

,image_filename,class,class_numeric
,<chr>,<chr>,<dbl>
1618,/kaggle/input/leukemia-images/Original/Pre/WBC-Malignant-Pre-129.jpg,Pre,2
1098,/kaggle/input/leukemia-images/Original/Early/WBC-Malignant-Early-594.jpg,Early,1
3176,/kaggle/input/leukemia-images/Original/Pro/WBC-Malignant-Pro-724.jpg,Pro,3
104,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-104.jpg,Benign,0
514,/kaggle/input/leukemia-images/Original/Early/WBC-Malignant-Early-010.jpg,Early,1
3161,/kaggle/input/leukemia-images/Original/Pro/WBC-Malignant-Pro-709.jpg,Pro,3


,image_filename,class,class_numeric
,<chr>,<chr>,<dbl>
6,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-006.jpg,Benign,0
8,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-008.jpg,Benign,0
15,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-015.jpg,Benign,0
19,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-019.jpg,Benign,0
21,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-021.jpg,Benign,0
28,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-028.jpg,Benign,0


,image_filename,class,class_numeric
,<chr>,<chr>,<dbl>
1,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-001.jpg,Benign,0
3,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-003.jpg,Benign,0
9,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-009.jpg,Benign,0
12,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-012.jpg,Benign,0
17,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-017.jpg,Benign,0
18,/kaggle/input/leukemia-images/Original/Benign/WBC-Benign-018.jpg,Benign,0


In [3]:
# Calculate class weights using only the training dataset
n_samples <- nrow(train_df)  
n_classes <- length(unique(train_df$class))

# Get class frequencies from training dataset
class_counts <- table(train_df$class_numeric)

# Calculate balanced weights
class_weights <- n_samples / (n_classes * class_counts)

# Convert to a named list with numeric indices as names
class_weights <- as.list(class_weights)
names(class_weights) <- as.character(0:(n_classes-1))

# Print diagnostics
cat("\nClass distribution in training dataset:\n")
print(table(train_df$class))

cat("\nFinal Class Weights (based on training dataset):\n")
print(class_weights)

# Validation steps
cat("\nValidation:\n")
cat("Total samples in training dataset:", n_samples, "\n")
cat("Sum of class counts:", sum(class_counts), "\n")
cat("Number of classes:", n_classes, "\n")
cat("Maximum weight ratio:", max(unlist(class_weights)) / min(unlist(class_weights)), "\n\n")

# Additional validation checks
cat("Verification of weight calculation:\n")
for(i in 0:(n_classes-1)) {
  class_name <- classes[i+1]
  weight <- class_weights[[as.character(i)]]
  count <- class_counts[as.character(i)]
  cat(sprintf("%s: count=%d, weight=%.4f, count*weight=%.4f\n", 
              class_name, count, weight, count*weight))
}

# Features extraction function 
extract_features <- function(image_path, bins = 16, feature_params = NULL) {
  # Load and resize the image using imager
  image <- load.image(image_path)
  image <- resize(image, img_width, img_height)
  
  # Convert image to array and split into color channels
  red_channel <- as.vector(image[,,1,1])
  green_channel <- as.vector(image[,,1,2])
  blue_channel <- as.vector(image[,,1,3])
  
  # Color histogram features for each channel
  red_hist <- hist(red_channel, breaks = bins, plot = FALSE)$counts / length(red_channel)
  green_hist <- hist(green_channel, breaks = bins, plot = FALSE)$counts / length(green_channel)
  blue_hist <- hist(blue_channel, breaks = bins, plot = FALSE)$counts / length(blue_channel)
  
  # Convert to grayscale for texture and edge features
  gray_image <- 0.299 * red_channel + 0.587 * green_channel + 0.114 * blue_channel
  
  # Texture features: contrast, skewness, and kurtosis
  contrast <- var(gray_image)
  skewness <- e1071::skewness(gray_image)
  kurtosis <- e1071::kurtosis(gray_image)
  
  # Statistical descriptors
  mean_intensity <- mean(gray_image)
  sd_intensity <- sd(gray_image)
  max_intensity <- max(gray_image)
  min_intensity <- min(gray_image)
  
  # Edge detection using gradient magnitude approximation (Sobel-like)
  gray_img_cimg <- as.cimg(gray_image, x = img_width, y = img_height)
  grad_x <- imgradient(gray_img_cimg, "x")
  grad_y <- imgradient(gray_img_cimg, "y")
  edge_magnitude <- sqrt(grad_x^2 + grad_y^2)  # Approximate edge strength
  edge_intensity_mean <- mean(edge_magnitude)
  edge_intensity_sd <- sd(edge_magnitude)
  
  # Shape features using segmentation (using EBImage)
  gray_img_matrix <- matrix(gray_image, nrow = img_height, ncol = img_width)
  binary_image <- gray_img_matrix > 0.5  # Apply threshold to get binary mask
  binary_image <- as.Image(binary_image)  # Convert to EBImage object
  
  # Label the binary image
  labeled_image <- bwlabel(binary_image)
  
  # Compute shape features
  shape_features <- computeFeatures.shape(labeled_image)
  
  # Shape descriptors - Mean values of area, perimeter, and circularity
  area_mean <- mean(shape_features[,"s.area"], na.rm = TRUE)
  perimeter_mean <- mean(shape_features[,"s.perimeter"], na.rm = TRUE)
  circularity_mean <- mean((4 * pi * shape_features[,"s.area"]) / (shape_features[,"s.perimeter"]^2), na.rm = TRUE)
  
  # Combine all features into a single vector
  feature_vector <- c(red_hist, green_hist, blue_hist, contrast, skewness, kurtosis, 
                      mean_intensity, sd_intensity, max_intensity, min_intensity,
                      edge_intensity_mean, edge_intensity_sd,
                      area_mean, perimeter_mean, circularity_mean)
  
  return(feature_vector)
}

# Function to analyze feature lengths across training data
get_feature_params <- function(train_image_paths, sample_size = NULL) {
  # If sample_size is provided, randomly sample images for efficiency
  if (!is.null(sample_size) && sample_size < length(train_image_paths)) {
    set.seed(123)  # for reproducibility
    image_paths <- sample(train_image_paths, sample_size)
  } else {
    image_paths <- train_image_paths
  }
  
  cat("Analyzing features from", length(image_paths), "training images...\n")
  
  # Extract features from all training images and analyze
  feature_lengths <- sapply(image_paths, function(path) {
    tryCatch({
      features <- extract_features(path)
      length(features)
    }, error = function(e) {
      warning(sprintf("Error processing image %s: %s", path, e$message))
      return(NA)
    })
  })
  
  # Analyze feature length distribution
  length_summary <- summary(feature_lengths)
  length_table <- table(feature_lengths)
  
  # Check for inconsistencies
  if (length(unique(feature_lengths[!is.na(feature_lengths)])) > 1) {
    warning("Inconsistent feature lengths detected in training data!")
    print(length_table)
    print(length_summary)
  }
  
  # Get most common feature length (mode)
  modal_length <- as.numeric(names(length_table)[which.max(length_table)])
  
  # Get feature names from a successful extraction
  sample_features <- NULL
  for (path in image_paths) {
    tryCatch({
      sample_features <- extract_features(path)
      if (length(sample_features) == modal_length) break
    }, error = function(e) {
      next
    })
  }
  
  params <- list(
    feature_length = modal_length,
    feature_names = paste0("feature_", seq_len(modal_length)),
    length_distribution = length_table,
    length_summary = length_summary,
    sample_size = length(image_paths),
    success_rate = sum(!is.na(feature_lengths)) / length(feature_lengths),
    consistent_length = length(unique(feature_lengths[!is.na(feature_lengths)])) == 1
  )
  
  # Print summary
  cat("\nFeature extraction analysis summary:\n")
  cat("Modal feature length:", params$feature_length, "\n")
  cat("Success rate:", sprintf("%.2f%%", params$success_rate * 100), "\n")
  cat("Length consistency:", ifelse(params$consistent_length, "Consistent", "Inconsistent"), "\n")
  
  if (!params$consistent_length) {
    cat("\nFeature length distribution:\n")
    print(params$length_distribution)
  }
  
  return(params)
}

# Modified feature extraction pipeline
set.seed(123)  # for reproducibility

# Step 1: Get feature parameters from training data
# Option to sample if training set is very large
sample_size <- if(nrow(train_df) > 1000) 1000 else NULL
feature_params <- get_feature_params(train_df$image_filename, sample_size)

# Enhanced feature extraction function with validation
extract_and_validate_features <- function(image_paths, feature_params, dataset_name) {
  features_list <- lapply(seq_along(image_paths), function(i) {
    path <- image_paths[i]
    
    if (i %% 100 == 0) {
      cat(sprintf("Processing %s image %d of %d\n", 
                 dataset_name, i, length(image_paths)))
    }
    
    tryCatch({
      features <- extract_features(path)
      
      # Validate feature length
      if (length(features) != feature_params$feature_length) {
        
        # Handle inconsistent length
        if (length(features) < feature_params$feature_length) {
          # Pad with NA if shorter
          features <- c(features, rep(NA, feature_params$feature_length - length(features)))
        } else {
          # Truncate if longer
          features <- features[1:feature_params$feature_length]
        }
      }
      
      return(features)
      
    }, error = function(e) {
      warning(sprintf("Error processing %s image %s: %s", 
                     dataset_name, path, e$message))
      return(rep(NA, feature_params$feature_length))
    })
  })
  
  # Convert to matrix/data frame
  features_matrix <- do.call(rbind, features_list)
  colnames(features_matrix) <- feature_params$feature_names
  
  # Report statistics about NA values
  na_counts <- colSums(is.na(features_matrix))
  if (any(na_counts > 0)) {
    cat("\nNA counts in", dataset_name, "dataset:\n")
    print(na_counts[na_counts > 0])
  }
  
  return(features_matrix)
}

# Extract features for each dataset
cat("\nExtracting training features...\n")
train_features <- extract_and_validate_features(
  train_df$image_filename, 
  feature_params,
  "training"
)

cat("\nExtracting validation features...\n")
val_features <- extract_and_validate_features(
  val_df$image_filename, 
  feature_params,
  "validation"
)

cat("\nExtracting test features...\n")
test_features <- extract_and_validate_features(
  test_df$image_filename, 
  feature_params,
  "test"
)

# Ensure that class labels are factors with consistent levels
train_df$class <- factor(train_df$class, levels = c("Benign", "Early", "Pre", "Pro"))
test_df$class <- factor(test_df$class, levels = c("Benign", "Early", "Pre", "Pro"))
val_df$class <- factor(val_df$class, levels = c("Benign", "Early", "Pre", "Pro"))

train_data <- cbind(as.data.frame(train_features), class_numeric = train_df$class_numeric)
validation_data <- cbind(as.data.frame(val_features), class_numeric = val_df$class_numeric)
test_data <- cbind(as.data.frame(test_features), class_numeric = test_df$class_numeric)

train_data[is.na(train_data)] <- 0
validation_data[is.na(validation_data)] <- 0
test_data[is.na(test_data)] <- 0

# Standardize the features
preprocess_params <- preProcess(train_data[, -ncol(train_data)], method = c("center", "scale"))
train_data_scaled <- predict(preprocess_params, train_data)
validation_data_scaled <- predict(preprocess_params, validation_data)
test_data_scaled <- predict(preprocess_params, test_data)

# Convert class_numeric to factor with proper labels for training and testing
train_data_scaled$class <- factor(train_data_scaled$class_numeric, 
                                levels = 0:3, 
                                labels = classes)
validation_data_scaled$class <- factor(validation_data_scaled$class_numeric, 
                                     levels = 0:3, 
                                     labels = classes)
test_data_scaled$class <- factor(test_data_scaled$class_numeric, 
                                levels = 0:3, 
                                labels = classes)

library(caret)
library(e1071)


# Function to identify NZV features from training data
identify_nzv_features <- function(train_data) {
  nzv <- nearZeroVar(train_data, saveMetrics = TRUE)
  nzv_features <- rownames(nzv)[nzv$zeroVar | nzv$nzv]
  
  # Log the features being removed and their properties
  cat("Features removed due to near-zero variance:\n")
  for(feature in nzv_features) {
    cat(sprintf("Feature: %s, Variance: %.6f\n", 
                feature, 
                var(train_data[[feature]], na.rm = TRUE)))
  }
  
  return(nzv_features)
}

# Function to check for differences in feature variance between datasets
check_feature_variance <- function(train_data, val_data, test_data, nzv_features) {
  warning_features <- list()
  
  for(feature in nzv_features) {
    if(feature %in% names(val_data) && feature %in% names(test_data)) {
      train_var <- var(train_data[[feature]], na.rm = TRUE)
      val_var <- var(val_data[[feature]], na.rm = TRUE)
      test_var <- var(test_data[[feature]], na.rm = TRUE)
      
      # Check if feature has significant variance in validation or test
      if(val_var > 0.1 || test_var > 0.1) {  # Threshold can be adjusted
        warning_features[[feature]] <- list(
          train_variance = train_var,
          val_variance = val_var,
          test_variance = test_var
        )
      }
    }
  }
  
  return(warning_features)
}

# Apply the fixed NZV filtering process
# First, identify NZV features from training data only
nzv_features <- identify_nzv_features(train_data_scaled[, -ncol(train_data_scaled)])

# Check for potential issues in validation and test sets
variance_warnings <- check_feature_variance(
  train_data_scaled[, -ncol(train_data_scaled)],
  validation_data_scaled[, -ncol(validation_data_scaled)],
  test_data_scaled[, -ncol(test_data_scaled)],
  nzv_features
)

# Print warnings if any features show different behavior in val/test
if(length(variance_warnings) > 0) {
  cat("\nWARNING: Some features show different variance patterns in validation/test sets:\n")
  print(variance_warnings)
  cat("\nConsider keeping these features in the model.\n")
}

# Remove NZV features from all datasets
train_data_filtered <- train_data_scaled[, !names(train_data_scaled) %in% nzv_features]
validation_data_filtered <- validation_data_scaled[, !names(validation_data_scaled) %in% nzv_features]
test_data_filtered <- test_data_scaled[, !names(test_data_scaled) %in% nzv_features]

head(train_data_filtered)
head(validation_data_filtered)
head(test_data_filtered)


Class distribution in training dataset:



Benign  Early    Pre    Pro 
   323    631    617    515 



Final Class Weights (based on training dataset):


$`0`
[1] 1.614551

$`1`
[1] 0.8264659

$`2`
[1] 0.8452188

$`3`
[1] 1.012621




Validation:


...

Processing test image 100 of 518
Processing test image 200 of 518
Processing test image 300 of 518
Processing test image 400 of 518
Processing test image 500 of 518

NA counts in test dataset:
feature_52 feature_53 feature_54 feature_55 feature_56 feature_57 feature_58 
         1          9         26         58         82        101        117 
feature_59 feature_60 feature_61 feature_62 
       121        130        138        169 


Features removed due to near-zero variance:


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,⋯,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,class_numeric,class
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0.03163656,2.6236003,3.1068061,1.5554515,0.59113280,0.1519493,-0.1634713,-0.3242494,-0.50210272,-0.6200269,⋯,-0.08694183,-0.1310055,-0.1227530,-0.1248577,-0.1868748,-0.4557918,-0.4828846,-0.1807713,2,Pre
2,-0.06400671,-0.2602499,-0.3348147,-0.4120166,-0.46574645,-0.5371078,-0.6452848,-0.6382741,-0.09371064,0.3021439,⋯,-0.14496976,-0.1334710,-0.1227402,-0.1248448,6.1239650,-0.3783744,-0.4827923,-0.1807713,1,Early
3,-0.25529325,-0.2883850,-0.3505538,-0.4146364,-0.46873620,-0.5939824,-0.7390973,-0.8789369,-0.94534665,-0.7833372,⋯,-0.14500556,-0.1335207,-0.1222629,-0.1247699,-0.1868666,-0.4557878,2.2416456,0.1394499,3,Pro
4,0.22292310,-0.1195743,-0.1721771,-0.1395576,-0.02176747,0.8902248,2.9614860,3.2212775,1.79365058,0.6953313,⋯,-0.14539556,-0.1335620,-0.1227530,-0.1248577,-0.1868748,-0.4557918,-0.4828846,-0.1807713,0,Benign
5,-0.25529325,-0.2602499,-0.3453074,-0.3989176,-0.29234052,0.2481986,2.2455966,2.0868520,0.49340883,0.1228576,⋯,-0.14539556,-0.1335620,-0.1227530,-0.1248577,-0.1868748,-0.4557918,-0.4828846,-0.1807713,1,Early
6,-0.25529325,-0.2883850,-0.3505538,-0.4146364,-0.46873620,-0.5939824,-0.7390973,-0.8798315,-0.95696393,-0.9164706,⋯,-0.14535618,-0.1331336,-0.1226554,-0.1248451,-0.1868664,2.4129871,-0.4106924,-0.1803437,3,Pro


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,⋯,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,class_numeric,class
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,-0.2552933,-0.2883850,-0.2875973,-0.32294344,-0.41492057,-0.4167963,-0.4458195,-0.07463998,0.2360414,0.5506596,⋯,-0.1450724,-0.1335044,-0.1222631,-0.1247701,-0.1868660,-0.4557879,-0.2232348,-0.1476746,0,Benign
2,-0.2552933,-0.2883850,-0.3505538,-0.41463635,-0.42986936,-0.5403891,-0.6644117,-0.67495502,-0.7737684,-0.9404346,⋯,-0.1420055,-0.1331637,-0.1226909,-0.1244115,-0.1868187,-0.4557888,-0.4828796,1.4970126,0,Benign
3,-0.2552933,-0.2883850,-0.3505538,-0.35962060,-0.09800628,-0.2407039,-0.4385331,-0.60696107,-0.7496402,-0.6413282,⋯,-0.1446751,-0.1331992,-0.1226744,-0.1244115,-0.1868187,-0.4557866,-0.4828787,1.4430539,0,Benign
4,-0.2552933,-0.2883850,-0.2036554,-0.19981295,-0.17723486,-0.3675779,-0.5797072,-0.81541621,-0.8988775,-0.9155831,⋯,-0.1428318,-0.1331798,-0.1226843,-0.1244115,-0.1868188,-0.4557872,-0.4828788,1.7570324,0,Benign
5,-0.2552933,-0.2883850,-0.3505538,-0.41463635,-0.45378742,-0.5217954,-0.5605804,-0.61590764,-0.7317674,-0.8179519,⋯,-0.1450164,-0.1335027,-0.1222629,-0.1247687,-0.1868646,-0.4557864,2.1455912,0.1394499,0,Benign
6,-0.2552933,-0.1758445,0.1950690,0.07264601,-0.24300952,-0.3850778,-0.4667679,-0.53986177,-0.4002281,-0.1913373,⋯,-0.1459526,-0.1330058,-0.1223547,-0.1247846,-0.1865923,-0.4557667,-0.4828798,-0.1807476,0,Benign


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,⋯,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,class_numeric,class
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,-0.2552933,-0.28838501,-0.35055381,-0.41201656,-0.4522925,-0.5480452,-0.6206932,-0.6168023,-0.5986155,-0.7682487,⋯,-0.1450000,-0.1335072,-0.1222629,-0.1247696,-0.1868679,-0.4557870,-0.1544160,-0.1392661,0,Benign
2,-0.2552933,0.04923647,-0.05675691,0.03858864,0.3220546,-0.2385164,-0.5642236,-0.7411597,-0.7281929,-0.6581918,⋯,-0.1461094,-0.1325405,-0.1223414,-0.1247806,-0.1865922,-0.4557668,-0.4828790,-0.1807418,0,Benign
3,-0.2552933,-0.28838501,-0.33481469,-0.23387032,-0.1607912,-0.3708591,-0.4549275,-0.5872786,-0.7978966,-0.8330403,⋯,-0.1435910,-0.1332172,-0.1226898,-0.1244115,-0.1868191,-0.4557877,-0.4828799,1.4547207,0,Benign
4,-0.2552933,-0.28838501,-0.35055381,-0.37795919,-0.4223950,-0.5272642,-0.6589469,-0.6883749,-0.7693002,-0.9892502,⋯,-0.1453479,-0.1331336,-0.1226558,-0.1248443,-0.1868644,-0.1450028,-0.4740961,-0.1775628,0,Benign
5,-0.2552933,-0.23211477,-0.30858282,-0.37795919,-0.4149206,-0.4507022,-0.6197824,-0.7313184,-0.7594702,-0.7726865,⋯,-0.1437448,-0.1331838,-0.1226887,-0.1244115,-0.1868190,-0.4557875,-0.4828799,2.1722095,0,Benign
6,-0.2552933,-0.28838501,-0.35055381,-0.08716165,-0.1084704,-0.4539835,-0.6270688,-0.7921551,-0.8175566,-0.9280088,⋯,-0.1453869,-0.1345130,-0.1199973,-0.1244625,-0.1868346,-0.4556643,-0.4828608,-0.1807514,0,Benign


In [4]:
# Set the number of folds for cross-validation
num_folds <- 5

# Create a 5-fold cross-validation partition for the training set
set.seed(123)  # For reproducibility
folds <- createFolds(train_data_filtered$class, k = num_folds, list = TRUE)

# Define the tune grid for SVM hyperparameters (sigma and C)
tune_grid <- expand.grid(
  sigma = 10^(-3:3),
  C = 10^(-3:3)
)

# Store performance results for each fold
cv_results <- data.frame(
  fold = integer(),
  train_accuracy = numeric(),
  valid_accuracy = numeric(),
  sigma = numeric(),
  C = numeric(),
  stringsAsFactors = FALSE
)

# Loop through each fold to perform training and validation
for (fold_idx in 1:num_folds) {
  cat("Training fold", fold_idx, "of", num_folds, "\n")
  
  # Get the training data for the current fold
  train_indices <- unlist(folds[[fold_idx]])
  train_fold <- train_data_filtered[train_indices, ]
  
  # Create class weights for the training fold
  class_weights_fold <- sapply(train_fold$class_numeric, 
                             function(x) class_weights[[as.character(x)]])
  
  # Initialize tracking for best model
  best_valid_accuracy <- -Inf
  best_train_accuracy <- -Inf
  best_svm_model <- NULL
  best_sigma <- NULL
  best_C <- NULL
  
  # Grid search over hyperparameters
  for (sigma_val in tune_grid$sigma) {
    for (C_val in tune_grid$C) {
      # Train the SVM model
      tryCatch({
        model <- svm(
          class ~ ., 
          data = train_fold[, !names(train_fold) %in% c("class_numeric")],
          kernel = "radial",
          cost = C_val,
          gamma = sigma_val,
          weights = class_weights_fold,
          probability = TRUE,
          scale = FALSE  # Disable internal scaling since data is already scaled
        )
        
        # Make predictions on the external validation set
        valid_predictions <- predict(model, validation_data_filtered)
        valid_accuracy <- confusionMatrix(valid_predictions, 
                                        validation_data_filtered$class)$overall['Accuracy']
        
        # Make predictions on training fold
        train_predictions <- predict(model, train_fold)
        train_accuracy <- confusionMatrix(train_predictions, 
                                        train_fold$class)$overall['Accuracy']
        
        # Track the best model based on validation accuracy
        if (valid_accuracy > best_valid_accuracy) {
          best_valid_accuracy <- valid_accuracy
          best_train_accuracy <- train_accuracy
          best_svm_model <- model
          best_sigma <- sigma_val
          best_C <- C_val
        }
      }, error = function(e) {
        cat("Error with sigma =", sigma_val, "and C =", C_val, ":", e$message, "\n")
      })
    }
  }
  
  # Store the results for the current fold
  cv_results <- rbind(cv_results, data.frame(
    fold = fold_idx,
    train_accuracy = best_train_accuracy,
    valid_accuracy = best_valid_accuracy,
    sigma = best_sigma,
    C = best_C,
    stringsAsFactors = FALSE
  ))
  
  cat("Best model for fold", fold_idx, ":\n",
      "sigma =", best_sigma, "\n",
      "C =", best_C, "\n",
      "Training accuracy:", best_train_accuracy, "\n",
      "Validation accuracy:", best_valid_accuracy, "\n\n")
}

# Print cross-validation results summary
cat("Cross-validation results:\n")
print(cv_results)

# Calculate summary statistics
mean_train_accuracy <- mean(cv_results$train_accuracy)
sd_train_accuracy <- sd(cv_results$train_accuracy)
mean_valid_accuracy <- mean(cv_results$valid_accuracy)
sd_valid_accuracy <- sd(cv_results$valid_accuracy)

cat("\nSummary Statistics:\n")
cat("Mean training accuracy:", mean_train_accuracy, "±", sd_train_accuracy, "\n")
cat("Mean validation accuracy:", mean_valid_accuracy, "±", sd_valid_accuracy, "\n")

# Select best hyperparameters based on validation performance
best_params <- cv_results[which.max(cv_results$valid_accuracy), ]

# Train final model on full training data
final_svm_model <- svm(
  class ~ ., 
  data = train_data_filtered[, !names(train_data_filtered) %in% c("class_numeric")],
  kernel = "radial",
  cost = best_params$C,
  gamma = best_params$sigma,
  weights = sapply(train_data_filtered$class_numeric, 
                  function(x) class_weights[[as.character(x)]]),
  probability = TRUE,
  scale = FALSE
)

# Evaluate final model on all datasets
evaluate_model <- function(model, data, set_name) {
  predictions <- predict(model, data)
  cm <- confusionMatrix(predictions, data$class)
  cat("\n", set_name, "set evaluation:\n")
  print(cm)
  return(cm)
}

# Final evaluations
train_cm <- evaluate_model(final_svm_model, train_data_filtered, "Training")
validation_cm <- evaluate_model(final_svm_model, validation_data_filtered, "Validation")
test_cm <- evaluate_model(final_svm_model, test_data_filtered, "Test")

cat("\nClass-Specific Statistics:\n")
print(test_cm$byClass)

cat("\nConfusion Matrix Mode:\n")
print(test_cm$mode)

cat("\nAdditional Arguments Passed to confusionMatrix():\n")
print(test_cm$dots)

# Print overall accuracy results
accuracy_test <- test_cm$overall['Accuracy']
accuracy_train <- train_cm$overall['Accuracy']
cat("Final Test Accuracy:", accuracy_test, "\n")
cat("Final Train Accuracy:", accuracy_train, "\n")

# Evaluating misclassification cost 
# Define the cost matrix (replace with your actual costs)
cost_matrix <- matrix(
  c(0, 10, 20, 30,   # Benign -> Benign, Early, Pre, Pro
    10, 0, 15, 25,    # Early -> Benign, Early, Pre, Pro
    20, 15, 0, 10,    # Pre -> Benign, Early, Pre, Pro
    30, 25, 10, 0),   # Pro -> Benign, Early, Pre, Pro
  nrow = 4, byrow = TRUE
)
rownames(cost_matrix) <- c("Benign", "Early", "Pre", "Pro")
colnames(cost_matrix) <- c("Benign", "Early", "Pre", "Pro")

# Extract confusion matrix data
cm_table <- test_cm$table
classes <- rownames(cm_table)

visualize_cost_confusion_matrix <- function(test_cm, cost_matrix) {
  # Extract the confusion matrix from caret's result
  cm_table <- test_cm$table
  
  # Create cost-weighted matrix
  cost_weighted_cm <- matrix(0, 
                              nrow = nrow(cost_matrix), 
                              ncol = ncol(cost_matrix))
  
  for (i in 1:nrow(cm_table)) {
    for (j in 1:ncol(cm_table)) {
      if (i != j) {
        cost_weighted_cm[i, j] <- cm_table[i, j] * cost_matrix[i, j]
      }
    }
  }
  
  # Set row and column names
  rownames(cost_weighted_cm) <- rownames(cost_matrix)
  colnames(cost_weighted_cm) <- colnames(cost_matrix)
  
  return(cost_weighted_cm)
}

# Usage
cost_weighted_matrix <- visualize_cost_confusion_matrix(test_cm, cost_matrix)
cat("\nCost-Weighted Misclassification Matrix:\n")
print(cost_weighted_matrix)

# Initialize total misclassification cost
total_cost <- 0

# Calculate total misclassification cost by iterating over confusion matrix entries
for (i in 1:nrow(cm_table)) {
  for (j in 1:ncol(cm_table)) {
    if (i != j) {  # Misclassifications (i != j)
      total_cost <- total_cost + cm_table[i, j] * cost_matrix[i, j]
    }
  }
}

# Output the total misclassification cost
cat("Total Misclassification Cost:", total_cost, "\n")

# Calculate average cost per sample
total_samples <- sum(cm_table)
avg_cost_per_sample <- total_cost / total_samples
cat("Average Cost Per Sample:", avg_cost_per_sample, "\n")

# Calculate cost-sensitive accuracy
# Compute the maximum possible cost (if all instances were misclassified to the most costly class)
max_possible_cost <- sum(cost_matrix) * (total_samples / nrow(cost_matrix))
cost_sensitive_accuracy <- 1 - (total_cost / max_possible_cost)
cat("Cost-Sensitive Accuracy:", cost_sensitive_accuracy, "\n")

# Using cost matrix for cost-weighted F1 calculation
f1_scores <- test_cm$byClass[, "F1"]
weights <- 1 / (diag(cost_matrix) + 1)  # Add 1 to avoid divide-by-zero
weights <- weights / sum(weights)
weighted_f1 <- sum(f1_scores * weights, na.rm = TRUE)
cat("Weighted F1-score", weighted_f1, "\n")

# Save final model and results
save(final_svm_model, cv_results, train_cm, validation_cm, test_cm,
     file = "final_svm_model.RData")

Training fold 1 of 5 
Best model for fold 1 :
 sigma = 0.01 
 C = 10 
 Training accuracy: 0.9807692 
 Validation accuracy: 0.8865031 

Training fold 2 of 5 
Best model for fold 2 :
 sigma = 0.01 
 C = 100 
 Training accuracy: 0.9976134 
 Validation accuracy: 0.8742331 

Training fold 3 of 5 
Best model for fold 3 :
 sigma = 0.01 
 C = 100 
 Training accuracy: 0.9952038 
 Validation accuracy: 0.8895706 

Training fold 4 of 5 
Best model for fold 4 :
 sigma = 0.01 
 C = 1000 
 Training accuracy: 1 
 Validation accuracy: 0.8757669 

Training fold 5 of 5 
Best model for fold 5 :
 sigma = 0.01 
 C = 100 
 Training accuracy: 0.9976077 
 Validation accuracy: 0.9064417 



Cross-validation results:


          fold train_accuracy valid_accuracy sigma    C
Accuracy     1      0.9807692      0.8865031  0.01   10
Accuracy1    2      0.9976134      0.8742331  0.01  100
Accuracy2    3      0.9952038      0.8895706  0.01  100
Accuracy3    4      1.0000000      0.8757669  0.01 1000
Accuracy4    5      0.9976077      0.9064417  0.01  100



Summary Statistics:


Mean training accuracy: 0.9942388 ± 0.007718304 


...

       Benign Early Pre Pro
Benign      0    50  20  90
Early     100     0  45   0
Pre         0    30   0   0
Pro         0     0  10   0


Total Misclassification Cost: 345 


Average Cost Per Sample: 0.6660232 


Cost-Sensitive Accuracy: 0.9878905 


Weighted F1-score 0.9447882 
